In [1]:
# !python -m spacy download en_core_web_md

In [2]:
import pandas as pd
from features import extract_dependency_features, extract_features
from utils import recombine_sentences

/home/hasan-sh/.virtualenvs/tm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-24 14:43:32 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-02-24 14:43:33 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| constituency | wsj       |
| ner          | ontonotes |

2023-02-24 14:43:33 INFO: Use device: cpu
2023-02-24 14:43:33 INFO: Loading: tokenize
2023-02-24 14:43:33 INFO: Loading: pos
2023-02-24 14:43:34 INFO: Loading: lemma
2023-02-24 14:43:34 INFO: Loading: depparse
202

In [ ]:

train_file = '../data/en_ewt-up-train.conllu'
test_file = '../data/en_ewt-up-test.conllu'

def create_sentences(file_name):
    # Open and read the input file
    with open(file_name, 'r', encoding='utf-8') as f:
        content = f.read()

    # Split the file into individual documents, each separated by a blank line
    sentences = {}
    for doc_i, doc in enumerate(content.split('\n\n')):
        doc = doc.split('\n')
        sentence = []
        for line in doc:
            # Skip lines starting with '#' (comment lines)
            if line and line[0] != '#':
                line = line.split('\t')
                sentence.append(line[1])
        sentences[doc_i] = ' '.join(sentence)
    return sentences

In [89]:

def extract_features(sentence, sent_id):
    """
    Extracts various linguistic features from a given sentence using the Stanza library.

    Args:
        sentence (str): The input sentence to extract features from.

    Returns:
        pandas.DataFrame: A DataFrame of features extracted from the sentence, with the following columns:
            - token (str): The original form of each token in the sentence.
            - pos (str): The part-of-speech tag for each token.
            - lemma (str): The lemma of each token.
            - ner (str): The named entity label for each token (if any).
            - stemming (str): The stemmed form of each token (using the Snowball stemmer).
            - pos_bigram (str): A string representing the part-of-speech bigram for each token and its successor.
            - token_bigram (str): A string representing the token bigram for each token and its successor.
    """
    # Load the English model
    # nlp = stanza.Pipeline('en', processors='tokenize,pos,lemma,ner')
    
    # Process the sentence
    doc = nlp(sentence)
    
    # Initialize lists to store the feature values
    tokens = []
    ner_tags = []
    pos_bigrams = []
    token_bigrams = []
    
    # Extract features for each token in the sentence
    for i, sent in enumerate(doc.sentences):
        ne = get_named_entities(sentences[sent_id])
            
        for j, word in enumerate(sent.words):
            # Add token to list
            tokens.append(word.text)
            
            # Add named entity label to list if it exists, otherwise add an empty string
            named_entity = list(filter(lambda x: x[0] == word.text, ne))
            if named_entity:
                ner_tags.append(named_entity[0][1])
            else:
                ner_tags.append('_')
            
            
            # Add part-of-speech bigram to list
            if j < len(sent.words) - 1:
                pos_bigrams.append(f"{word.upos}_{sent.words[j+1].upos}")
            else:
                pos_bigrams.append('')
                
            # Add token bigram to list
            if j < len(sent.words) - 1:
                token_bigrams.append(f"{word.text}_{sent.words[j+1].text}")
            else:
                token_bigrams.append('')
    
    return {
        'token': tokens,
        'ner': ner_tags,
        'pos_bigram': pos_bigrams,
        'token_bigram': token_bigrams
    }


In [ ]:
result = [extract_features(sent, i) for i, sent in sentences.items()]

In [ ]:
df_train = pd.DataFrame(result).explode(list(result[0].keys()))

In [ ]:
train_file = '../data/train.tsv'
test_file = '../data/test.tsv'

df_train_1 = pd.read_csv(train_file, delimiter='\t')

In [ ]:
sentences = create_sentences('../data/en_ewt-up-test.conllu')

In [ ]:
result = [extract_features(sent, i) for i, sent in sentences.items()]

In [ ]:
df_test = pd.DataFrame(result).explode(list(result[0].keys()))

In [ ]:
df_train.to_csv('../data/

In [ ]:
for t in nlp('How does it work?'):
    print(t.ner)

In [ ]:

# all_fe = pd.concat([basic_fe, dependency_fe], axis=1)
df = pd.DataFrame([])

for i, s in sentences.items():
    basic_fe = extract_features(s)
    dependency_fe = extract_dependency_features(s)
    
    df = pd.concat([df, pd.concat([basic_fe, dependency_fe], axis=1)])
    print(basic_fe)
    break

In [ ]:
0881122112 # gza health care

In [ ]:
df